# AND(empiar11415)

In [ ]:
import mrcfile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from medpy.filter.smoothing import anisotropic_diffusion

In [ ]:
from my_google_auth import DriveHandler
service = DriveHandler.get_drive_service()
handler = DriveHandler.DriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'

In [ ]:
with mrcfile.open("input.mrc", permissive=True) as mrc:
    vol = mrc.data.astype(np.float32)

In [ ]:
vol = (vol - vol.min()) / (vol.max() - vol.min() + 1e-8)

denoised = anisotropic_diffusion(
    vol,
    niter=20,     # number of iterations (10–30) (More iterations = stronger smoothing)
    kappa=30,     # conduction coefficient (Edge detection sensitivity) (20–40) (Larger → smoother but weaker edges)
    gamma=0.1,    # time step (Step size) (stability) (0.05–0.15) (Must be ≤0.25)
    option=1,     # PM1 model (Conduction model) (1 or 2) (1 = sharper edges preserved)
    voxelspacing=None,  # use if voxel spacing differs per axis (Voxel geometry) (None or (sx,sy,sz)) (Use if voxels are anisotropic)
)

In [ ]:
denoised = anisotropic_diffusion(
    vol,          # [0.0, 1.0]!
    niter=12,     # number of iterations (10–30) (More iterations = stronger smoothing)
    kappa=15,     # conduction coefficient (Edge detection sensitivity) (20–40) (Larger → smoother but weaker edges)
    gamma=0.07,   # time step (Step size) (stability) (0.05–0.15) (Must be ≤0.25)
    option=1,     # PM1 model (Conduction model) (1 or 2) (1 = sharper edges preserved)
    voxelspacing=None,  # use if voxel spacing differs per axis (Voxel geometry) (None or (sx,sy,sz)) (Use if voxels are anisotropic)
)

In [ ]:
denoised.shape

In [ ]:
Z_dim = vol.shape[0]
Z2 = Z_dim//2

In [ ]:
with mrcfile.new(f"denoised.mrc", overwrite=True) as mrc:
    mrc.set_data(denoised.astype(np.float32))
    mrc.data

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoised.mrc',
    drive_file_name=f'empiar11415__AND.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
def read_MRC(file_path):
    return mrcfile.read(file_path)

In [ ]:
denoised = read_MRC("denoised.mrc")

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(vol[7][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("FlowDenoising")
plt.imshow(denoised[7][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(vol[7][::-1, :] - denoised[7][::-1, :], cmap="gray")

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 0:500, 0:500], cmap="gray")
plt.savefig("denoised.pdf", bbox_inches='tight')

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoised.pdf',
    drive_file_name='empiar11415__AND.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)